# Demand Neural Network and Revenue Optimization

Yudi Wang 

Computational Science & Engineering, Harvard University


In [1]:
#import libraries and packages
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
#translate from the matlab code
import xlrd
import pandas as pd

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from keras.models import Sequential
from keras.layers import Dense, Activation

#translate from the matlab code
import xlrd
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
#reading and setting data
NumSKU=49
NumVar=13
ID=[1138263,1139362,1139363,1141061,1142731,1143640,1144140,
        1148001,1148010,1148081,1162466,1162467,1162557,1162558,
        1162559,1163152,1163153,1164313,1164961,1164962,1165757,
        1166153,1166984,1166998,1167021,1167087,1167847,1167918,
        1170236,1170372,1170739,1173299,1174241,1174242,1174243,
        1174244,1174275,1174293,1174299,1174313,1174314,1174315,
        1174339,1174340,1175687,1175833,1175835,1175950,1177151]

#train data
new_tv_info = pd.read_excel('C:/Users/wyd15/Downloads/Television.xlsx', sheet_name='new_tv_info') 
tv_sim = pd.read_excel('Television.xlsx', sheet_name='Similarity')
#tv_sim.head()

In [3]:
print(len(new_tv_info))

#reorder
cols = list(new_tv_info)
cols.insert(3, cols.pop(cols.index('Price'))) #changed price locations
new_tv_info = new_tv_info.loc[:, cols]

new_tv_info.head()

24697


,ID,Date,SalesQuantity,Price,SalesQuantityLag1,SalesQuantityLag7,SalesQuantityLag14,Discount,InventoryAvailability,WeekOfYear,...,Var75,Var76,Var77,Var78,Var79,Var80,Var81,Var82,Var83,Var84
0,1,2016-01-01,2,2626.27000,1,1,1,1.01,0.93,1,...,1779.66000,1.01,2329.659900,1.01,2533.055050,1.01,2541.530000,1.01,1753.387550,1.01
1,1,2016-01-02,6,2651.69398,2,1,1,1.01,0.93,1,...,1880.51008,1.01,2365.113317,1.01,2555.651667,1.01,2794.132285,1.01,1842.372600,1.01
2,1,2016-01-03,5,2774.57500,6,1,1,1.01,0.93,1,...,1851.69608,1.01,2355.083980,1.01,2725.285017,1.01,2835.449192,1.01,1789.243869,1.01
3,1,2016-01-04,6,2795.76000,5,1,1,1.01,0.93,2,...,1876.27505,1.01,2468.837450,1.01,2965.260000,1.01,2721.716275,1.01,1905.935100,1.01
4,1,2016-01-05,2,2795.76000,6,1,1,1.01,0.93,2,...,1876.27505,1.01,2468.837450,1.01,2626.273333,1.01,3473.720000,1.01,1905.925200,1.01


# Training demand neural network

In [5]:
beta = 0.0001 #0.0001
alpha = 0.112 #learning_rate 0.112

tf.reset_default_graph()
x1=tf.placeholder(shape=(None, 18), dtype=tf.float32)
x2=tf.placeholder(shape=(None, 61), dtype=tf.float32)
y =tf.placeholder(shape=(None, ), dtype=tf.float32)

#
W1_1 = tf.get_variable('W1_1', shape=(18, 79))
W1_2 = tf.get_variable('W1_2', shape=(61, 79))
b1 = tf.get_variable('b1', shape=(1, 79))
    
o1_1 = tf.matmul(x1, W1_1)
o1_2 = tf.matmul(x2, W1_2)
o1 = o1_1 + o1_2 + b1
o1 = tf.nn.sigmoid(o1)
W2_1 = tf.get_variable('W2_1', shape=(61, 1))
W2_2 = tf.get_variable('W2_2', shape=(79, 1))
b2 = tf.get_variable('b2', shape=(1, 1))

o2_1 = tf.matmul(x2, W2_1)
o2_2 = tf.matmul(o1, W2_2)
o2 = o2_1 + o2_2 + b2
o2 = tf.reshape(o2, (-1, ))
saver = tf.train.Saver([W1_1, W1_2, b1, W2_1, W2_2, b2])

#add regularization on all the variables 
loss = tf.losses.mean_squared_error(y, o2)
#loss = tf.losses.cosine_distance(y, o2, axis=0)
train_var  = tf.trainable_variables() 
#add l2 regularization exclude all the bias
lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in train_var if 'b' not in v.name ]) * beta #
#loss with regularization
loss= tf.reduce_mean(tf.losses.mean_squared_error(y, o2)+lossL2) 

optimizer = tf.train.AdamOptimizer(learning_rate=0.11) #tuned learning_rate
train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

#train-test_split
msk = np.random.rand(len(new_tv_info)) < 0.85 #train_test split
tv_train = new_tv_info[msk]
tv_test = new_tv_info[~msk]
batch_size=50

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(419):
        x1_train = tv_train.iloc[step*batch_size:step*batch_size+batch_size, 66:84].values 
        x2_train = np.concatenate((tv_train.iloc[step*batch_size:step*batch_size+batch_size, 3:11].values, tv_train.iloc[step*batch_size:step*batch_size+batch_size, 14:67].values), 1) 
        y_train = tv_train.iloc[step*batch_size:step*batch_size+batch_size, 2].values 
            
        x1_test = tv_test.iloc[:, 66:84].values 
        x2_test = np.concatenate((tv_test.iloc[:, 3:11].values, tv_test.iloc[:, 14:67].values), 1) 
        y_test = tv_test.iloc[:, 2].values 
            
        _train_loss, _ = sess.run([loss, train_op],feed_dict={x1: x1_train,
                                              x2: x2_train,
                                              y: y_train})
            
        _test_loss = sess.run([loss],feed_dict={x1: x1_test,
                                              x2: x2_test,
                                              y: y_test})
        # predict
        D_predict = sess.run([o2], feed_dict={x1:x1_test,x2:x2_test})
        para_pred = sess.run([W1_1, W1_2, b1, W2_1, W2_2, b2], feed_dict={x1:x1_test,x2:x2_test})
        #results, _ = sess.run([output]) #print parameters, W1_1 etc..
            
        if step % 100==0:
            saver.save(sess, 'C:/Users/wyd15/Desktop/tv_model/tv_modelslack.ckpt', global_step=step)
            print("iter:%d, train_loss: %f, test_loss: %f"%(step, _train_loss,  _test_loss[0]))
            #print('test prediction parameters results:', D_predict)
            #print('predicted parameters:', para_pred)
    print('\n last test prediction parameters results:', step, _train_loss,  _test_loss[0], '\n predicted test demand', y_test,'\n', D_predict)
    paras=para_pred
                #print('output:', )

iter:0, train_loss: 4208.549316, test_loss: 369652.843750
iter:100, train_loss: 91.331482, test_loss: 2076.224609
iter:200, train_loss: 46.772224, test_loss: 65.661057
iter:300, train_loss: 186.604828, test_loss: 28.385403
iter:400, train_loss: 2.813472, test_loss: 29.611162

 last test prediction parameters results: 418 46.9461 30.4771 
 predicted test demand [ 2 10  7 ..., 10 24 17] 
 [array([  0.07984164,  -1.48110962,   3.28382373, ...,   6.84973192,
         7.4528861 ,  17.14572144], dtype=float32)]


# Parameters for TV NN

In [5]:
import numpy as np
np.set_printoptions(threshold=1500)
import pickle
para_names=['W1_1', 'W1_2', 'b1', 'W2_1', 'W2_2', 'b2']
for i in range(len(para_names)):
    print(' ===== printing paramter: '+ para_names[i] +' ===== ')
    print(para_pred[i])

 ===== printing paramter: W1_1 ===== 
[[  5.10159552e-01  -5.60055196e-01  -5.71182251e-01   5.58091700e-01
   -5.86390734e-01  -5.39967120e-01  -4.66522366e-01  -2.43708193e-01
   -2.36894131e-01  -7.19668269e-01  -5.42470992e-01  -5.74124157e-01
    4.14336622e-01   5.86916208e-01  -6.40039325e-01   5.35342932e-01
   -5.18492699e-01  -5.75787127e-01  -5.32501340e-01   6.68643415e-01
    4.99356449e-01  -1.14664781e+00   5.12747407e-01   6.59732614e-04
    6.06913447e-01   6.48585260e-01   5.57623267e-01  -2.84736395e-01
   -5.72863877e-01  -4.94941980e-01  -1.03350461e+00   5.97277403e-01
   -6.84463739e-01  -6.36129856e-01  -4.76181954e-01   7.25725651e-01
   -6.06885433e-01  -5.86100340e-01  -5.67891777e-01   7.20831573e-01
   -6.08471632e-01  -4.87495422e-01  -1.07454491e+00  -5.69918990e-01
    5.11958480e-01   5.34504533e-01   8.31178248e-01   6.27393723e-01
    5.61331809e-01   5.62491775e-01   7.46560335e-01  -5.15101254e-01
   -5.31714916e-01   1.90579724e+00   6.37645721e-01

# Optimization nn for focal product price

simulation note:
- ignored constant vector for the optimization, i.e. set as zero vector
- only optimize over one focal product price

In [6]:
import time 

#optimize the loss funciton by adding negative sign
start_time= time.time()
tf.reset_default_graph()
p_f = tf.get_variable('p_f', shape=(1, 1)) #corresponding index 0 in 
W1_1 = tf.constant(paras[0], dtype=tf.float32, name='W1_1' )
W1_2_1 = tf.constant(paras[1][0, :], dtype=tf.float32, shape=[1, 79], name='W1_2_1')
W1_2_2 = tf.constant(paras[1][1:, :], dtype=tf.float32, name='W1_2_2')
b1 = tf.constant(paras[2], dtype=tf.float32, name='b1' )
W2_1_1 = tf.constant(paras[3][0, :], dtype=tf.float32, shape=[1, 1], name='W2_1_1')
W2_1_2 = tf.constant(paras[3][1:, :], dtype=tf.float32, name='W2_1_2')
W2_2 = tf.constant(paras[4], dtype=tf.float32, name='W2_2' )
b2 = tf.constant(paras[5], dtype=tf.float32, name='b2' )
p_sub =  tf.placeholder(shape=(None, 18), dtype=tf.float32)
p_t =  tf.placeholder(shape=(None, 60), dtype=tf.float32) #the other part except for the price

D = tf.matmul(tf.nn.sigmoid(tf.matmul(p_f, W1_2_1)+ tf.matmul(p_t, W1_2_2) + tf.matmul(p_sub, W1_1)+b1), W2_2)+tf.matmul(p_f, W2_1_1)+ tf.matmul(p_t, W2_1_2)+b2
    
p_loss = tf.reduce_mean(-p_f*D) #self defined loss

###parameters
batch_size = 50
learning_rate = 0.5
trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)  #why not SGD??
gradients = trainer.compute_gradients(p_loss)
optimizer = trainer.apply_gradients(gradients) #??
#optimizer = tf.contrib.opt.ScipyOptimizerInterface(p_loss, method='L-BFGS-B', options={'maxiter': 1000})

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(419):
        #parameters 
        x1_train = tv_train.iloc[step*batch_size:step*batch_size+batch_size, 66:84].values 
        x1_test = tv_test.iloc[:, 66:84].values
        x2_train = np.concatenate((tv_train.iloc[step*batch_size:step*batch_size+batch_size, 4:11].values, tv_train.iloc[step*batch_size:step*batch_size+batch_size, 14:67].values), 1) 
        x2_test = np.concatenate((tv_test.iloc[:, 4:11].values, tv_test.iloc[:, 14:67].values), 1) 
        
        loss_tr, _ = sess.run([p_loss, optimizer], feed_dict={p_sub: x1_train, p_t: x2_train})
        _test_loss = sess.run([p_loss], feed_dict={p_sub: x1_test, p_t: x2_test})
        
        # predict
        D_pred_tr = sess.run([D], feed_dict={p_sub:x1_train, p_t: x2_train})
        D_pred_te = sess.run([D], feed_dict={p_sub:x1_test, p_t: x2_test})
        pf_pred_tr = sess.run([p_f], feed_dict={p_sub:x1_train, p_t: x2_train})
        pf_pred_te = sess.run([p_f], feed_dict={p_sub:x1_test, p_t: x2_test})
        
        if step % 200 == 0:
            print()
            print("iter:%d, train_loss: %f, test_loss: %f"%(step, loss_tr,  _test_loss[0]))
            print("simulated train_revenue: %f, simulated test_revenue: %f"%(-loss_tr,  -_test_loss[0]))
            print('train D prediction parameters results:', D_pred_tr[0][0])
            print('test D prediction parameters results:', D_pred_te[0][0])
            print('train p_f prediction parameters results:', pf_pred_tr[0][0])
            print('test p_f prediction parameters results:', pf_pred_te[0][0])
            print()
print('\n last test prediction parameters results:', step, -loss_tr,  -_test_loss[0], 
    '\n predicted mean demand and price for one batch', np.mean(D_pred_te[0]), pf_pred_te[0])

end_time=time.time()
print('trained total elasped time:', end_time-start_time)


iter:0, train_loss: 0.596367, test_loss: -3.796283
simulated train_revenue: -0.596367, simulated test_revenue: 3.796283
train D prediction parameters results: [ 4.24965668]
test D prediction parameters results: [ 7.47465944]
train p_f prediction parameters results: [ 0.44520485]
test p_f prediction parameters results: [ 0.44520485]


iter:200, train_loss: -1448.016357, test_loss: -758.030090
simulated train_revenue: 1448.016357, simulated test_revenue: 758.030090
train D prediction parameters results: [ 24.57897758]
test D prediction parameters results: [ 7.22606802]
train p_f prediction parameters results: [ 91.56659698]
test p_f prediction parameters results: [ 91.56659698]


iter:400, train_loss: -976.682861, test_loss: -1542.622681
simulated train_revenue: 976.682861, simulated test_revenue: 1542.622681
train D prediction parameters results: [ 4.77977562]
test D prediction parameters results: [ 6.94996881]
train p_f prediction parameters results: [ 192.77095032]
test p_f predictio

# Conclusion on trained result:

For optimize over one price, we got the optimized result of revenue 55384.5, corresponding predicted demand around 143 and price as 387.63. 

This method is proved to be fast, only took about 1 minute to train over the whole neural network, whicle flexible in terms of able to train on more prices only need to adjust the parameter setting for the second neural networ 'p_f, p_sub and p_t'

# Next Step

- Check focal product price manually;


- Check the Focal Product Elasticity
- Check the Cross Price Elasticity
- Check the Optimization algorithm

- Try optimize over more price, other competitor prices in other columns (need to adjust columns)


## Defination of Price Elasticity:

Price Elasticity of Demand = :

change in Quantity Demanded% / Change in Price%. 

If a small change in price is accompanied by a large change in quantity demanded, the product is said to be elastic (or responsive to price changes). On the other hand, a product is deemed inelastic if a large change in price is accompanied by a small amount of change in quantity demanded.

Checking with calculations:

In [7]:
W1_1 = paras[0]
W1_2_1 = paras[1][0, :].reshape((1, 79))
W1_2_2 = paras[1][1:, :]
b1 = paras[2]
W2_1_1 = paras[3][0, :]
W2_1_2 = paras[3][1:, :]
W2_2 = paras[4]
b2 = paras[5]

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def demand_fun(p_f, p_t, p_sub):
    p1 = np.matmul(sigmoid(np.matmul(p_f, W1_2_1) + np.matmul(p_t, W1_2_2) + np.matmul(p_sub, W1_1)+b1), W2_2)
    p2 = np.matmul(p_f, W2_1_1).reshape((len(p_f), 1)) 
    p3 = np.matmul(p_t, W2_1_2)+b2
    D = p1 + p2 + p3
    return D

p_f_train = tv_train.iloc[:, 3].values.reshape((len(tv_train),1))
p_t_train = np.concatenate((tv_train.iloc[:, 4:11].values, tv_train.iloc[:, 14:67].values), 1).astype(float)
p_sub_train = tv_train.iloc[:, 66:84].values.astype(float)

p_f_test = tv_test.iloc[:, 3].values.reshape((len(tv_test),1))
p_t_test = np.concatenate((tv_test.iloc[:, 4:11].values, tv_test.iloc[:, 14:67].values), 1).astype(float)
p_sub_test = tv_test.iloc[:, 66:84].values.astype(float)

D_train = demand_fun(p_f_train, p_t_train, p_sub_train)
p_f_train = p_f_train*1.1
new_D_train = demand_fun(p_f_train, p_t_train, p_sub_train)

D_test = demand_fun(p_f_test, p_t_test, p_sub_test)
p_f_test = p_f_test*1.1
new_D_test = demand_fun(p_f_test, p_t_test, p_sub_test)

In [8]:
#save result to excel file
tr_result = pd.DataFrame(columns =['Price', 'Price * 1.1', 'True Sales', 'Pred Demand', 'Approx Pred Demand', 'New Pred Demand', 'Approx New Pred Demand', 'Elasticity'])
tr_result['Price'] = tv_train.iloc[:, 3].values
tr_result['Price * 1.1'] = tv_train.iloc[:, 3].values*1.1
tr_result['True Sales'] = tv_train.iloc[:, 2].values
tr_result['Pred Demand'] = D_train
tr_result['Approx Pred Demand'] = list(map(int, np.round(D_train)))
tr_result['New Pred Demand'] = new_D_train
tr_result['Approx New Pred Demand'] = list(map(int, np.round(new_D_train)))
tr_result['Elasticity'] = ((new_D_train-D_train)/D_train)/0.1
tr_result.to_csv('NN Elasticity Train Result.csv')

#save result to excel file
te_result = pd.DataFrame(columns =['Price', 'Price * 1.1', 'True Sales', 'Pred Demand', 'Approx Pred Demand', 'New Pred Demand', 'Approx New Pred Demand', 'Elasticity'])
te_result['Price'] = tv_test.iloc[:, 3].values
te_result['Price * 1.1'] = tv_test.iloc[:, 3].values*1.1
te_result['True Sales'] = tv_test.iloc[:, 2].values
te_result['Pred Demand'] = D_test
te_result['Approx Pred Demand'] = list(map(int, np.round(D_test)))
te_result['New Pred Demand'] = new_D_test
te_result['Approx New Pred Demand'] = list(map(int, np.round(new_D_test)))
te_result['Elasticity'] = ((new_D_test-D_test)/D_test)/0.1
te_result.to_csv('NN Elasticity Test Result.csv')
print(tr_result.columns)
tr_result.head()

Index(['Price', 'Price * 1.1', 'True Sales', 'Pred Demand',
       'Approx Pred Demand', 'New Pred Demand', 'Approx New Pred Demand',
       'Elasticity'],
      dtype='object')


,Price,Price * 1.1,True Sales,Pred Demand,Approx Pred Demand,New Pred Demand,Approx New Pred Demand,Elasticity
0,2626.27000,2888.897000,2,-2.913951,-3,-3.630433,-4,2.458800
1,2651.69398,2916.863378,6,-2.287149,-2,-3.010567,-3,3.162969
2,2774.57500,3052.032500,5,0.602375,1,-0.154567,0,-12.565958
3,2795.76000,3075.336000,6,-0.954706,-1,-1.717427,-2,7.989074
4,2795.76000,3075.336000,2,0.358935,0,-0.403787,0,-21.249580


In [28]:
def get_elasticity(tr_name= 'NN Elasticity Train Result.csv', te_name= 'NN Elasticity Test Result.csv'):
    '''
    function for getting elasticity
    Args:
        tr_name:  train save file name, default 'NN Elasticity Train Result.csv'
        te_name:  test save file name, default 'NN Elasticity Test Result.csv'
    
    Returns:
        df_elasticity: 
    
    Usecase:
        
        
    '''
    W1_1 = paras[0]
    W1_2_1 = paras[1][0, :].reshape((1, 79))
    W1_2_2 = paras[1][1:, :]
    b1 = paras[2]
    W2_1_1 = paras[3][0, :]
    W2_1_2 = paras[3][1:, :]
    W2_2 = paras[4]
    b2 = paras[5]

    def sigmoid(x):
        return (1 / (1 + np.exp(-x)))

    def demand_fun(p_f, p_t, p_sub):
        p1 = np.matmul(sigmoid(np.matmul(p_f, W1_2_1) + np.matmul(p_t, W1_2_2) + np.matmul(p_sub, W1_1)+b1), W2_2)
        p2 = np.matmul(p_f, W2_1_1).reshape((len(p_f), 1)) 
        p3 = np.matmul(p_t, W2_1_2)+b2
        D = p1 + p2 + p3
        return D
    
    #train
    p_f_train = tv_train.iloc[:, 3].values.reshape((len(tv_train),1))
    p_t_train = np.concatenate((tv_train.iloc[:, 4:11].values, tv_train.iloc[:, 14:67].values), 1).astype(float)
    p_sub_train = tv_train.iloc[:, 66:84].values.astype(float)
    #test
    p_f_test = tv_test.iloc[:, 3].values.reshape((len(tv_test),1))
    p_t_test = np.concatenate((tv_test.iloc[:, 4:11].values, tv_test.iloc[:, 14:67].values), 1).astype(float)
    p_sub_test = tv_test.iloc[:, 66:84].values.astype(float)
    #cal train
    D_train = demand_fun(p_f_train, p_t_train, p_sub_train)
    p_f_train = p_f_train*1.1
    new_D_train = demand_fun(p_f_train, p_t_train, p_sub_train)
    #cal test
    D_test = demand_fun(p_f_test, p_t_test, p_sub_test)
    p_f_test = p_f_test*1.1
    new_D_test = demand_fun(p_f_test, p_t_test, p_sub_test)

    #save result to train excel file
    tr_result = pd.DataFrame(columns =['Price', 'Price * 1.1', 'True Sales', 'Pred Demand', 'Approx Pred Demand', 'New Pred Demand', 'Approx New Pred Demand', 'Elasticity'])
    tr_result['Price'] = tv_train.iloc[:, 3].values
    tr_result['Price * 1.1'] = tv_train.iloc[:, 3].values*1.1
    tr_result['True Sales'] = tv_train.iloc[:, 2].values
    tr_result['Pred Demand'] = D_train
    tr_result['Approx Pred Demand'] = list(map(int, np.round(D_train)))
    tr_result['New Pred Demand'] = new_D_train
    tr_result['Approx New Pred Demand'] = list(map(int, np.round(new_D_train)))
    tr_result['Elasticity'] = ((new_D_train-D_train)/D_train)/0.1
    tr_result.to_csv(tr_name)
    
    #save result to excel file
    te_result = pd.DataFrame(columns =['Price', 'Price * 1.1', 'True Sales', 'Pred Demand', 'Approx Pred Demand', 'New Pred Demand', 'Approx New Pred Demand', 'Elasticity'])
    te_result['Price'] = tv_test.iloc[:, 3].values
    te_result['Price * 1.1'] = tv_test.iloc[:, 3].values*1.1
    te_result['True Sales'] = tv_test.iloc[:, 2].values
    te_result['Pred Demand'] = D_test
    te_result['Approx Pred Demand'] = list(map(int, np.round(D_test)))
    te_result['New Pred Demand'] = new_D_test
    te_result['Approx New Pred Demand'] = list(map(int, np.round(new_D_test)))
    te_result['Elasticity'] = ((new_D_test-D_test)/D_test)/0.1
    te_result.to_csv(te_name)
    
    print('===== Finished !! ===== ')
    return tr_result, te_result

tr_ela, te_ela = get_elasticity(tr_name= 'NN Elasticity Train Result.csv', te_name= 'NN Elasticity Test Result.csv')

===== Finished !! ===== 


Check
1. check NN with Yasaman result
2. divide price;
2. try LBFGS on NN;
3. 

## Check the NN optimization algorith and LBFGS algorithm

An L-BFGS solver is a true second order method in that it estimates the curvature of the parameter space via an approximation of the Hessian. So if your parameter space has plenty of long, nearly-flat valleys then L-BFGS would likely perform well. It has the downside of additional costs in performing a rank-two update to the Hessian approximation at every step. While this is reasonably fast, it does begin to add up, particularly as the input space grows. This may account for the fact that ADAM outperforms L-BFGS for you as you get more data.

ADAM is a first order method that attempts to compensate for the fact that it doesn't estimate the curvature by adapting the step-size in every dimension. In some sense, this is similar to constructing a diagonal Hessian at every step, but they do it cleverly by simply using past gradients. In this way it is still a first order method, though it has the benefit of acting as though it is second order. The estimate is cruder than that of the L-BFGS in that it is only along each dimension and doesn't account for what would be the off-diagonals in the Hessian. If your Hessian is nearly singular then these off-diagonals may play an important ro^le in the curvature and ADAM is likely to underperform relative the BFGS.

In [23]:
# https://www.tensorflow.org/api_docs/python/tf/contrib/opt/ScipyOptimizerInterface
# https://stackoverflow.com/questions/50988466/using-l-bfgs-optimizer-with-tensorflow-estimator-api
# https://stackoverflow.com/questions/47385084/do-tensorflow-scipyoptimizerinterface-l-bfgs-and-others-work-with-auto-differe
# https://stackoverflow.com/questions/33720645/why-is-this-tensorflow-implementation-vastly-less-successful-than-matlabs-nn
#trial

import time 
from tensorflow.contrib.opt import ScipyOptimizerInterface 

#optimize the loss funciton by adding negative sign
start_time= time.time()
tf.reset_default_graph()

p_f = tf.get_variable('p_f', shape=(1, 1)) #corresponding index 0 in 

W1_1 = tf.constant(paras[0], dtype=tf.float32, name='W1_1' )
W1_2_1 = tf.constant(paras[1][0, :], dtype=tf.float32, shape=[1, 79], name='W1_2_1')
W1_2_2 = tf.constant(paras[1][1:, :], dtype=tf.float32, name='W1_2_2')
b1 = tf.constant(paras[2], dtype=tf.float32, name='b1' )
W2_1_1 = tf.constant(paras[3][0, :], dtype=tf.float32, shape=[1, 1], name='W2_1_1')
W2_1_2 = tf.constant(paras[3][1:, :], dtype=tf.float32, name='W2_1_2')
W2_2 = tf.constant(paras[4], dtype=tf.float32, name='W2_2' )
b2 = tf.constant(paras[5], dtype=tf.float32, name='b2' )

p_sub =  tf.placeholder(shape=(None, 18), dtype=tf.float32 , name="p_sub")
p_t =  tf.placeholder(shape=(None, 60), dtype=tf.float32, name= 'p_t') #the other part except for the price

D = tf.matmul(tf.nn.sigmoid(tf.matmul(p_f, W1_2_1)+ tf.matmul(p_t, W1_2_2) + tf.matmul(p_sub, W1_1)+b1), W2_2)+tf.matmul(p_f, W2_1_1)+ tf.matmul(p_t, W2_1_2)+b2
    
p_loss = tf.reduce_mean(-p_f*D) #self defined loss

###parameters
learning_rate = 0.5 #lbfgs optimization
optimizer = tf.contrib.opt.ScipyOptimizerInterface(p_loss, var_to_bounds={p_f: (np.min(new_tv_info.iloc[:, 3]), np.max(new_tv_info.iloc[:, 3]))}, method='L-BFGS-B', options={'maxiter': 1000})

with tf.Session() as sess:
    init = tf.global_variables_initializer() 
    x1_train = tv_train.iloc[:, 66:84].values 
    x1_test = tv_test.iloc[:, 66:84].values
    x2_train = np.concatenate((tv_train.iloc[:, 4:11].values, tv_train.iloc[:, 14:67].values), 1) 
    x2_test = np.concatenate((tv_test.iloc[:, 4:11].values, tv_test.iloc[:, 14:67].values), 1) 
    sess.run(init)
    optimizer.minimize(sess, feed_dict={p_sub: x1_train, p_t: x2_train}) 
    ret=sess.run(p_f)
    print(ret)
     
end_time=time.time()
print('trained total elasped time:', end_time-start_time)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -22437.767578
  Number of iterations: 6
  Number of functions evaluations: 8
[[ 4086.68237305]]
trained total elasped time: 0.7696733474731445


In [33]:
import time 
from tensorflow.contrib.opt import ScipyOptimizerInterface 

#optimize the loss funciton by adding negative sign
start_time= time.time()
tf.reset_default_graph()

p_f = tf.get_variable('p_f', shape=(100, 1)) #corresponding index 0 in 

W1_1 = tf.constant(paras[0], dtype=tf.float32, name='W1_1' )
W1_2_1 = tf.constant(paras[1][0, :], dtype=tf.float32, shape=[1, 79], name='W1_2_1')
W1_2_2 = tf.constant(paras[1][1:, :], dtype=tf.float32, name='W1_2_2')
b1 = tf.constant(paras[2], dtype=tf.float32, name='b1' )
W2_1_1 = tf.constant(paras[3][0, :], dtype=tf.float32, shape=[1, 1], name='W2_1_1')
W2_1_2 = tf.constant(paras[3][1:, :], dtype=tf.float32, name='W2_1_2')
W2_2 = tf.constant(paras[4], dtype=tf.float32, name='W2_2' )
b2 = tf.constant(paras[5], dtype=tf.float32, name='b2' )

p_sub =  tf.placeholder(shape=(None, 18), dtype=tf.float32 , name="p_sub")
p_t =  tf.placeholder(shape=(None, 60), dtype=tf.float32, name= 'p_t') #the other part except for the price

D = tf.matmul(tf.nn.sigmoid(tf.matmul(p_f, W1_2_1)+ tf.matmul(p_t, W1_2_2) + tf.matmul(p_sub, W1_1)+b1), W2_2)+tf.matmul(p_f, W2_1_1)+ tf.matmul(p_t, W2_1_2)+b2
#D = tf.nn.sigmoid(tf.matmul(p_f, W1_2_1)+ tf.matmul(p_t, W1_2_2) +tf.matmul(p_sub, W1_1) + b1)

p_loss = tf.reduce_mean(-p_f*D) #self defined loss

###parameters
batch_size = 100
learning_rate = 0.5
optimizer = tf.contrib.opt.ScipyOptimizerInterface(p_loss, var_to_bounds={p_f: (np.min(new_tv_info.iloc[:, 3]), np.max(new_tv_info.iloc[:, 3]))}, method='L-BFGS-B', options={'maxiter': 1000} )
pf = [] 

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    for step in range(246):
        #parameters 
        
            x1_train = new_tv_info.iloc[step*batch_size:step*batch_size+batch_size, 66:84].values 
            #x1_test = tv_test.iloc[:, 66:84].values
            x2_train = np.concatenate((new_tv_info.iloc[step*batch_size:step*batch_size+batch_size, 4:11].values, new_tv_info.iloc[step*batch_size:step*batch_size+batch_size, 14:67].values), 1) 
            #x2_test = np.concatenate((tv_test.iloc[:, 4:11].values, tv_test.iloc[:, 14:67].values), 1) 
            sess.run(init)
            optimizer.minimize(sess, feed_dict={p_sub: x1_train, p_t: x2_train}) 
            res=sess.run(p_f)
            pf.append(res.flatten())

        
pf=np.array(pf).ravel()
end_time=time.time()
print('total predicted price',len(pf), 'trained total elasped time:', end_time-start_time)
opt_pf = pd.DataFrame({'opt_pf': pf})
tr_col_names = list(tr_ela.columns)
tr_ela = pd.concat([tr_ela, opt_pf], ignore_index=True, axis=1)
tr_ela.columns =  tr_col_names + ['opt_pf']
tr_ela.to_csv('NN Elasticity Train Result.csv')

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -32129.130859
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -45732.519531
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -12114.382812
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -10168.738281
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -7257.1

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -41404.863281
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -14894.932617
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -11267.424805
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -10863.919922
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -13965.

  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -23913.332031
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -12569.852539
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -9835.644531
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -11400.931641
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'

  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -48656.464844
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -23723.589844
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -9204.628906
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -15760.540039
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -9621.270508
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -22424.654297
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -13315.264648
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -13405.094727
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -6731.27

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -20246.898438
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -33392.734375
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -26730.892578
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -27791.142578
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -72005.4

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -21315.300781
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -10286.106445
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -21262.375000
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -14024.613281
  Number of iterations: 3
  Number of functions evaluations: 4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: -8368.1

In [34]:
all_result = pd.concat([tr_result, te_result], ignore_index=True)
all_result.head()
price = pd.DataFrame({'opt_price': pf})
col_names = np.hstack((list(all_result.columns), ['opt_price']))
all_result = pd.concat([all_result, price], ignore_index=True, axis=1)
all_result.columns= col_names
all_result.to_csv('NN Elasticity All Result.csv')
all_result.head()

,Price,Price * 1.1,True Sales,Pred Demand,Approx Pred Demand,New Pred Demand,Approx New Pred Demand,Elasticity,opt_price
0,2626.270,2888.8970,2,3.308201,3,2.955347,3,-1.066604,2544.271729
1,2774.575,3052.0325,5,6.408923,6,6.036143,6,-0.581657,2684.291016
2,2795.760,3075.3360,6,5.487435,5,5.111809,5,-0.684520,3772.348145
3,2795.760,3075.3360,2,6.743184,7,6.367558,6,-0.557045,3440.011719
4,2795.760,3075.3360,1,3.857496,4,3.481870,3,-0.973756,3907.334961


Next Step:

1. report on result so far;
2. multiple prices;
3. check lbfgs opt prices and send result to Yasaman;
4. construct the optimization function of multiple products, goal is for all 49 products. 